# Initial Retirement Assets
The main question when planning financially for retirement is "How much do I need to save?" The answer, of course, is highly dependent on a similar question, "How much money will I need in retirement?" 

Unfortunately, for many people, the single number estimate of how much money they will need is surprisingly large. Since retirement is expected to last for quite a long time, it's natural to wonder if it's reasonable to extend savings throughout retirement by investment.

Well-known rules of thumb answer this. Advisors suggest drawing down 3% or 4%, based on a [thirty-year old study](http://www.retailinvestor.org/pdf/Bengen1.pdf) by William Bergen. This rule of thumb can be easily manipulated to determine how much money one should have at the beginning of retirement. However, there are hidden market, interest-rate, and portfolio assumptions behind that simple rule of thumb.

Rather than rely on these hidden assumptions, you can calculate how much you'll need at the start of retirement using this notebook

## Your assumptions

In [47]:
years_of_retirement = 35
annual_expenses = 40_000 # in retirement, annualized

We'll also have to assume a real rate of return on our investment during retirement. Since we'll be playing around with different scenarios we use a list instead of single variable.

In [48]:
annual_returns = [.02] * years_of_retirement

## Deriving the calculation
In the simplest case, where the rate of return is constant, the calculation is fairly straightforward.

Let $P$ be the initial amount that we will have at retirement.
Let $n$ be the number of years that retirment will last, $a$ be the annual expenses, and let $r_1, r_2, \ldots, r_{n-1}$ be the returns in years $1, 2, \ldots, n-1$.

Then the total amount that we need is calculated as:

\begin{align}
na \leq\: &P\: + \\
& (P-a)r_1\: + \\
& (P-2a)r_2\: + (P-a)r_1 r_2\: + \\
& \vdots \\
& \big(P - (n-1)a)\big)r_{n-1}\: + \big(P - (n-2)a)\big)r_{n-2}r_{n-1}\: + \ldots + (P - 2a)r_1r_2 \ldots r_{n-2}\: +(P - a)r_1r_2 \ldots r_{n-1}
\end{align}

We can simplify this by grouping the $P$ and $a$ terms, and then we can solve for $P$.

\begin{equation}
na \leq\: P\: + P\Big(\sum_{i=1}^{n-1} \sum_{j=i}^{n-1} \prod_{k=i}^j r_k\Big) - a \Big( \sum_{i=1}^{n-1} i \sum_{j=i}^{n-1} \prod_{k=i}^j r_k\Big) \\
P \geq \frac{a \Big( n + \sum\limits_{i=1}^{n-1} i \sum\limits_{j=i}^{n-1} \prod\limits_{k=i}^j r_k\Big)}{1 + \Big(\sum\limits_{i=1}^{n-1} \sum\limits_{j=i}^{n-1} \prod\limits_{k=i}^j r_k\Big)}
\end{equation}

This will be very straigtforward to calculate.

We'll first calculate the repeated inner sum that is present in both numerator and denominator.

In [49]:
import numpy as np

def inner_sum(n, returns):
    # NOTE: We very quickly lose precision here!
    sums = np.zeros(n-1, dtype=np.longdouble)
    # Definitely could be vectorized, but these are such small loops. 
    # Refactor if using more frequent returns (e.g. daily)
    for i in range(1, n):
        for j in range(i, n):
            sums[i-1] += np.prod(returns[i:j+1])  # As above, dp would be useful if |returns| is large.
    return sums

In [50]:
def min_initial_amount(annual_expenses, num_years, returns):
    insum = inner_sum(num_years, returns)
    return annual_expenses * (num_years + np.dot(np.arange(1,num_years), insum)) / (1 + np.sum(insum))

## Calculations
We can now easily calculate the minimum required amount according to different assumptions

In [51]:
min_initial_amount(annual_expenses, years_of_retirement, annual_returns)

1113192.5273798648

In [52]:
# With a linearly decreasing rate of return (from 2% to .25%)
min_initial_amount(annual_expenses, years_of_retirement, np.linspace(.02, .0025, years_of_retirement -1))

1244101.2112565849

## Extensions
Since picking expected rates of return is mostly a stab in the dark, we can expand this calculation by first constructing a retirement portfolio, and then doing monte carlo simulations using randomly selected historical annual rates of return of such a portfolio. 